# Codebusters KPIs and Other Usefull Patterns

## import libraries

In [281]:
import pandas as pd
import numpy as np
from pathlib import Path
import glob

Import and clean `all_tickets_2022` data set 
- remove unnecessary columns
- rename columns

# Prepare data with all tickets

### Import all service request

Tickets are taken from the following jira filter:<br/> `project = CI AND issuetype in (standardIssueTypes(), "Expense Delivery") AND "Epic Link" is EMPTY AND "Case Number/s" is not EMPTY AND cf[14125] in ("Service Request (SR)") AND resolved is not EMPTY AND resolutiondate >= 2022-12-19

In [282]:
all_sr_origin = pd.read_csv('../../DataSets/MainTickets/all_service_requests.csv')
all_sr_origin

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_4400\3199886521.py:1: DtypeWarning: Columns (25,34,36,37,38,39,40,41,42,53,54,55,56,58,59,61,62,63,65,71,72,80,81,82,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,234,235,236,237,238,239,240,241,242,243,249,250,273,311,317,353,384,490,506,510,538,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614) have mixed types. Specify dtype option on import or set low_memory=False.
  all_sr_origin = pd.read_csv('../../DataSets/MainTickets/all_service_requests.csv')


,Summary,Issue key,Issue id,Issue Type,Status,Project key,Project name,Project type,Project lead,Project description,...,Comment.37,Comment.38,Comment.39,Comment.40,Comment.41,Comment.42,Comment.43,Comment.44,Comment.45,Comment.46
0,porsche - follow up CYTCI-9236: deploy in prod...,CYTCI-10248143,7829513,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"porsche - follow up CYTCI-9168, deploy in prod...",CYTCI-10248142,7829500,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Hugo boss – update AttachmentMaxSize to 5MB,CYTCI-10248129,7822247,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MFTBC - Remove fuel price on OCT 2022,CYTCI-10248128,7819124,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hugo boss – Add receitp type under County code...,CYTCI-10248127,7819049,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1343,Continental - Update the export format,CYTCI-5864,7651300,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1344,MSG - Limit the dropdown for internal employee...,CYTCI-5837,7651273,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1345,"ÖBB - Add new FDF ""fdf_edat"" to configuration",CYTCI-5749,7651186,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1346,Frankfurt School of Finance - new conditional ...,CYTCI-5527,7650968,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Utilites functions

In [283]:
def find_escalatetions(row, dataSet):
    label_columns = [col for col in dataSet if 'label' in col.lower()]
    isEscalated = False
    for column in label_columns:
        if 'escalated' in str(row[column]).lower():
            isEscalated = True    
    return isEscalated

# Analyze labels

## Find any escalated ticekt

In [284]:
all_sr_escalated_match = all_sr_origin
all_sr_escalated_match["Is Escalated"] = all_sr_escalated_match.apply(lambda row: find_escalatetions(row, all_sr_escalated_match), axis=1)
all_sr_escalated_match

,Summary,Issue key,Issue id,Issue Type,Status,Project key,Project name,Project type,Project lead,Project description,...,Comment.38,Comment.39,Comment.40,Comment.41,Comment.42,Comment.43,Comment.44,Comment.45,Comment.46,Is Escalated
0,porsche - follow up CYTCI-9236: deploy in prod...,CYTCI-10248143,7829513,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,"porsche - follow up CYTCI-9168, deploy in prod...",CYTCI-10248142,7829500,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,Hugo boss – update AttachmentMaxSize to 5MB,CYTCI-10248129,7822247,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,MFTBC - Remove fuel price on OCT 2022,CYTCI-10248128,7819124,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,Hugo boss – Add receitp type under County code...,CYTCI-10248127,7819049,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1343,Continental - Update the export format,CYTCI-5864,7651300,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1344,MSG - Limit the dropdown for internal employee...,CYTCI-5837,7651273,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1345,"ÖBB - Add new FDF ""fdf_edat"" to configuration",CYTCI-5749,7651186,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1346,Frankfurt School of Finance - new conditional ...,CYTCI-5527,7650968,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


# Filter the data

### Select from the dataset only columns necessary for the further analysis

In [285]:
all_sr = all_sr_escalated_match[['Issue key', 'Issue id', 'Status', 'Reporter', 'Created', 'Due Date', 'Resolved', 'Custom field (Customer/s)','Custom field (Type of Request)', 'Is Escalated']]
all_sr.rename({"Custom field (Customer/s)": "Customer", "Custom field (Type of Request)":"Type of Request"},axis=1, inplace=True)

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_4400\2470571524.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sr.rename({"Custom field (Customer/s)": "Customer", "Custom field (Type of Request)":"Type of Request"},axis=1, inplace=True)


In [286]:
all_sr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1348 entries, 0 to 1347
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Issue key        1348 non-null   object
 1   Issue id         1348 non-null   int64 
 2   Status           1348 non-null   object
 3   Reporter         1348 non-null   object
 4   Created          1348 non-null   object
 5   Due Date         1348 non-null   object
 6   Resolved         1348 non-null   object
 7   Customer         1348 non-null   object
 8   Type of Request  1348 non-null   object
 9   Is Escalated     1348 non-null   bool  
dtypes: bool(1), int64(1), object(8)
memory usage: 96.2+ KB


In [287]:
all_sr.isnull().sum()

Issue key          0
Issue id           0
Status             0
Reporter           0
Created            0
Due Date           0
Resolved           0
Customer           0
Type of Request    0
Is Escalated       0
dtype: int64

### Convert date table to `datetime` object

In [288]:
all_sr["Created"] = pd.to_datetime(all_sr["Created"])
all_sr["Resolved"] = pd.to_datetime(all_sr["Resolved"])

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_4400\2794271155.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sr["Created"] = pd.to_datetime(all_sr["Created"])
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_4400\2794271155.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sr["Resolved"] = pd.to_datetime(all_sr["Resolved"])


In [289]:
all_sr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1348 entries, 0 to 1347
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Issue key        1348 non-null   object        
 1   Issue id         1348 non-null   int64         
 2   Status           1348 non-null   object        
 3   Reporter         1348 non-null   object        
 4   Created          1348 non-null   datetime64[ns]
 5   Due Date         1348 non-null   object        
 6   Resolved         1348 non-null   datetime64[ns]
 7   Customer         1348 non-null   object        
 8   Type of Request  1348 non-null   object        
 9   Is Escalated     1348 non-null   bool          
dtypes: bool(1), datetime64[ns](2), int64(1), object(6)
memory usage: 96.2+ KB


In [290]:
all_sr.shape

(1348, 10)

In [291]:
all_sr["ticket_age"] = all_sr["Resolved"] - all_sr["Created"]
all_sr

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_4400\2540177896.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sr["ticket_age"] = all_sr["Resolved"] - all_sr["Created"]


,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age
0,CYTCI-10248143,7829513,Closed,askarja,2023-10-06 01:08:00,2023-10-13 00:00,2023-10-16 17:29:00,Porsche,Service Request (SR),False,10 days 16:21:00
1,CYTCI-10248142,7829500,Closed,askarja,2023-10-06 00:43:00,2023-10-13 00:00,2023-11-03 01:45:00,Porsche,Service Request (SR),False,28 days 01:02:00
2,CYTCI-10248129,7822247,Closed,yutaka.yasumi,2023-10-04 16:50:00,2023-10-14 00:00,2023-10-31 02:49:00,Hugo Boss,Service Request (SR),False,26 days 09:59:00
3,CYTCI-10248128,7819124,Closed,yutaka.yasumi,2023-10-04 05:35:00,2023-10-14 00:00,2023-10-04 07:14:00,MFTBC,Service Request (SR),False,0 days 01:39:00
4,CYTCI-10248127,7819049,Closed,yutaka.yasumi,2023-10-04 04:14:00,2023-10-13 00:00,2023-10-30 09:51:00,Hugo Boss,Service Request (SR),False,26 days 05:37:00
...,...,...,...,...,...,...,...,...,...,...,...
1343,CYTCI-5864,7651300,Closed,yutaka.yasumi,2021-10-19 09:20:00,2022-01-27 00:00,2022-03-07 09:30:00,Continental,Service Request (SR),False,139 days 00:10:00
1344,CYTCI-5837,7651273,Closed,ssanz,2021-10-13 12:23:00,2021-10-17 00:00,2022-03-07 12:40:00,msg systems,Service Request (SR),True,145 days 00:17:00
1345,CYTCI-5749,7651186,Closed,kmarscholl,2021-09-29 08:51:00,2021-12-30 00:00,2022-01-19 17:20:00,OBB,Service Request (SR),False,112 days 08:29:00
1346,CYTCI-5527,7650968,Closed,ngraf,2021-08-06 14:47:00,2021-08-12 00:00,2022-01-13 16:01:00,School of Finance,Service Request (SR),True,160 days 01:14:00


In [292]:
all_sr['Created'].sort_values(ascending=False)

13     2023-11-22 12:45:00
14     2023-11-17 11:49:00
15     2023-11-15 17:11:00
16     2023-11-15 11:41:00
17     2023-11-14 09:17:00
               ...        
1343   2021-10-19 09:20:00
1344   2021-10-13 12:23:00
1345   2021-09-29 08:51:00
1346   2021-08-06 14:47:00
1347   2021-08-04 09:02:00
Name: Created, Length: 1348, dtype: datetime64[ns]

**Summary**
- there is not SR ticket without `resolved` date
- there is no `null` value in any of the column
- date type columns converted to `datetime` object in order to make calculucations on these columns

### Select from the dataset only columns necessary for the further analysis

In [293]:
all_ir = all_sr_escalated_match[['Issue key', 'Issue id', 'Status', 'Reporter', 'Created', 'Due Date', 'Resolved', 'Custom field (Customer/s)','Custom field (Type of Request)', 'Is Escalated']]
all_ir.rename({"Custom field (Customer/s)": "Customer", "Custom field (Type of Request)":"Type of Request"},axis=1, inplace=True)

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_4400\1037294033.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ir.rename({"Custom field (Customer/s)": "Customer", "Custom field (Type of Request)":"Type of Request"},axis=1, inplace=True)


In [294]:
all_ir.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1348 entries, 0 to 1347
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Issue key        1348 non-null   object
 1   Issue id         1348 non-null   int64 
 2   Status           1348 non-null   object
 3   Reporter         1348 non-null   object
 4   Created          1348 non-null   object
 5   Due Date         1348 non-null   object
 6   Resolved         1348 non-null   object
 7   Customer         1348 non-null   object
 8   Type of Request  1348 non-null   object
 9   Is Escalated     1348 non-null   bool  
dtypes: bool(1), int64(1), object(8)
memory usage: 96.2+ KB


In [295]:
all_ir.isnull().sum()

Issue key          0
Issue id           0
Status             0
Reporter           0
Created            0
Due Date           0
Resolved           0
Customer           0
Type of Request    0
Is Escalated       0
dtype: int64

### Convert date table to `datetime` object

In [296]:
all_ir["Created"] = pd.to_datetime(all_ir["Created"])
all_ir["Resolved"] = pd.to_datetime(all_ir["Resolved"])

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_4400\119588641.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ir["Created"] = pd.to_datetime(all_ir["Created"])
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_4400\119588641.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ir["Resolved"] = pd.to_datetime(all_ir["Resolved"])


In [297]:
all_ir.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1348 entries, 0 to 1347
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Issue key        1348 non-null   object        
 1   Issue id         1348 non-null   int64         
 2   Status           1348 non-null   object        
 3   Reporter         1348 non-null   object        
 4   Created          1348 non-null   datetime64[ns]
 5   Due Date         1348 non-null   object        
 6   Resolved         1348 non-null   datetime64[ns]
 7   Customer         1348 non-null   object        
 8   Type of Request  1348 non-null   object        
 9   Is Escalated     1348 non-null   bool          
dtypes: bool(1), datetime64[ns](2), int64(1), object(6)
memory usage: 96.2+ KB


In [298]:
all_ir.shape

(1348, 10)

In [299]:
all_ir["ticket_age"] = all_ir["Resolved"] - all_ir["Created"]
all_ir

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_4400\2921873505.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ir["ticket_age"] = all_ir["Resolved"] - all_ir["Created"]


,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age
0,CYTCI-10248143,7829513,Closed,askarja,2023-10-06 01:08:00,2023-10-13 00:00,2023-10-16 17:29:00,Porsche,Service Request (SR),False,10 days 16:21:00
1,CYTCI-10248142,7829500,Closed,askarja,2023-10-06 00:43:00,2023-10-13 00:00,2023-11-03 01:45:00,Porsche,Service Request (SR),False,28 days 01:02:00
2,CYTCI-10248129,7822247,Closed,yutaka.yasumi,2023-10-04 16:50:00,2023-10-14 00:00,2023-10-31 02:49:00,Hugo Boss,Service Request (SR),False,26 days 09:59:00
3,CYTCI-10248128,7819124,Closed,yutaka.yasumi,2023-10-04 05:35:00,2023-10-14 00:00,2023-10-04 07:14:00,MFTBC,Service Request (SR),False,0 days 01:39:00
4,CYTCI-10248127,7819049,Closed,yutaka.yasumi,2023-10-04 04:14:00,2023-10-13 00:00,2023-10-30 09:51:00,Hugo Boss,Service Request (SR),False,26 days 05:37:00
...,...,...,...,...,...,...,...,...,...,...,...
1343,CYTCI-5864,7651300,Closed,yutaka.yasumi,2021-10-19 09:20:00,2022-01-27 00:00,2022-03-07 09:30:00,Continental,Service Request (SR),False,139 days 00:10:00
1344,CYTCI-5837,7651273,Closed,ssanz,2021-10-13 12:23:00,2021-10-17 00:00,2022-03-07 12:40:00,msg systems,Service Request (SR),True,145 days 00:17:00
1345,CYTCI-5749,7651186,Closed,kmarscholl,2021-09-29 08:51:00,2021-12-30 00:00,2022-01-19 17:20:00,OBB,Service Request (SR),False,112 days 08:29:00
1346,CYTCI-5527,7650968,Closed,ngraf,2021-08-06 14:47:00,2021-08-12 00:00,2022-01-13 16:01:00,School of Finance,Service Request (SR),True,160 days 01:14:00


**Summary**
- there is not ir ticket without `resolved` date
- there is no `null` value in any of the column
- date type columns converted to `datetime` object in order to make calculucations on these columns

In [300]:
def find_escalatetions(row, dataSet):
    label_columns = [col for col in dataSet if 'label' in col.lower()]
    isEscalated = False
    for column in label_columns:
        if 'escalated' in str(row[column]).lower():
            isEscalated = True    
    return isEscalated

# Prepare data with individuals tasks for service requests

## Import and clean `all_individual_tasks` data set 
- we are not able to downaload full data becuase of jira limitation, therefore we need to combine data from differen periods
- remove unnecessary columns
- replace missing `date` values with 0
- drop rows with "null" values

In [301]:
all_individual_tasks_path = glob.glob("../../DataSets/IndividualTasks/" + "*.csv")


all_individual_tasks_combined =pd.concat(map(lambda filename: pd.read_csv(filename,  encoding='unicode_escape').assign(source=filename), all_individual_tasks_path))
all_individual_tasks_combined

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_4400\2109289185.py:4: DtypeWarning: Columns (111,196,261,431,489) have mixed types. Specify dtype option on import or set low_memory=False.
  all_individual_tasks_combined =pd.concat(map(lambda filename: pd.read_csv(filename,  encoding='unicode_escape').assign(source=filename), all_individual_tasks_path))
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_4400\2109289185.py:4: DtypeWarning: Columns (21,26,39,40,41,80,83,94,267,296,306,438,439,497) have mixed types. Specify dtype option on import or set low_memory=False.
  all_individual_tasks_combined =pd.concat(map(lambda filename: pd.read_csv(filename,  encoding='unicode_escape').assign(source=filename), all_individual_tasks_path))
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_4400\2109289185.py:4: DtypeWarning: Columns (35,36,109,112,262,430,431,432,433,434,435,436,437,438,439,440,441,442,486,500,548,549) have mixed types. Specify dtype option on import or set low_memory=False.
  all_in

,Summary,Issue key,Issue id,Parent id,Issue Type,Status,Project key,Project name,Project type,Project lead,...,Inward issue link (Blocks).2,Comment.2,Comment.3,Comment.4,Comment.5,Inward issue link (Blocks).3,Inward issue link (Relates),Def of Done,Lifecycle Items,Comment.6
0,Review,CYTCI-10179980,7679692,7650050,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Clarify,CYTCI-10179977,7679691,7650034,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Explain,CYTCI-10179961,7679689,7650029,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Communicate,CYTCI-10179959,7679687,7649877,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Import in Production,CYTCI-10179958,7679686,7649877,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1263,Configure,CYTCI-10240611,7705909,7654080,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1264,Configure,CYTCI-10239936,7705523,7654026,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1265,Do,CYTCI-10239710,7705402,7654004,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1266,Review,CYTCI-10234353,7703060,7653656,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [302]:
all_individual_tasks_combined[['source', 'Created']].sort_values(by='Created', ascending=True)

,source,Created
3765,../../DataSets/IndividualTasks\from_20220901_t...,01/09/2022 02:07
3764,../../DataSets/IndividualTasks\from_20220901_t...,01/09/2022 03:11
3763,../../DataSets/IndividualTasks\from_20220901_t...,01/09/2022 04:22
3762,../../DataSets/IndividualTasks\from_20220901_t...,01/09/2022 09:07
3761,../../DataSets/IndividualTasks\from_20220901_t...,01/09/2022 09:29
...,...,...
2022,../../DataSets/IndividualTasks\from_20220901_t...,31/10/2022 16:17
2020,../../DataSets/IndividualTasks\from_20220901_t...,31/10/2022 16:38
2021,../../DataSets/IndividualTasks\from_20220901_t...,31/10/2022 16:38
2019,../../DataSets/IndividualTasks\from_20220901_t...,31/10/2022 19:29


### Reset indexex of a combined dataset </br>
Indexes of a new dataset are not appropriate, we need to apply `reset_index` method in order to match them with the number of rows

In [303]:
all_individual_tasks_combined.reset_index(inplace=True)
all_individual_tasks_combined

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_4400\4115877976.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_individual_tasks_combined.reset_index(inplace=True)


,index,Summary,Issue key,Issue id,Parent id,Issue Type,Status,Project key,Project name,Project type,...,Inward issue link (Blocks).2,Comment.2,Comment.3,Comment.4,Comment.5,Inward issue link (Blocks).3,Inward issue link (Relates),Def of Done,Lifecycle Items,Comment.6
0,0,Review,CYTCI-10179980,7679692,7650050,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Clarify,CYTCI-10179977,7679691,7650034,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Explain,CYTCI-10179961,7679689,7650029,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Communicate,CYTCI-10179959,7679687,7649877,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Import in Production,CYTCI-10179958,7679686,7649877,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33845,1263,Configure,CYTCI-10240611,7705909,7654080,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33846,1264,Configure,CYTCI-10239936,7705523,7654026,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33847,1265,Do,CYTCI-10239710,7705402,7654004,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33848,1266,Review,CYTCI-10234353,7703060,7653656,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [304]:
all_individual_tasks_combined.to_csv("all_individual_tasks_combined.csv")

### Filter columns and select only the ones needed for further calculations

In [305]:
all_individual_tasks = all_individual_tasks_combined[['Summary','Issue id', 'Parent id', 'Created', 'Resolved']]
all_individual_tasks

,Summary,Issue id,Parent id,Created,Resolved
0,Review,7679692,7650050,2021-02-27 14:46,2021-02-27 14:46
1,Clarify,7679691,7650034,2021-02-26 17:29,2021-02-26 17:29
2,Explain,7679689,7650029,2021-02-26 15:37,2021-02-26 15:37
3,Communicate,7679687,7649877,2021-02-26 15:19,2021-02-26 15:20
4,Import in Production,7679686,7649877,2021-02-26 15:19,2021-02-26 15:19
...,...,...,...,...,...
33845,Configure,7705909,7654080,2023-05-29 12:27,2023-11-17 12:15
33846,Configure,7705523,7654026,2023-05-15 12:39,2023-11-17 13:04
33847,Do,7705402,7654004,2023-05-11 10:58,2023-11-17 12:30
33848,Review,7703060,7653656,2023-03-03 12:53,2023-11-02 08:56


In [306]:
all_individual_tasks[all_individual_tasks['Parent id'] == 465210]

,Summary,Issue id,Parent id,Created,Resolved


### Convert date like columns to `datetime` object

In [307]:
all_individual_tasks["Created"] = pd.to_datetime(all_individual_tasks["Created"], dayfirst=True, format='mixed')
all_individual_tasks["Resolved"] = pd.to_datetime(all_individual_tasks["Resolved"], dayfirst=True,  format='mixed')
all_individual_tasks.info()

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_4400\2151095576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks["Created"] = pd.to_datetime(all_individual_tasks["Created"], dayfirst=True, format='mixed')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33850 entries, 0 to 33849
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Summary    33850 non-null  object        
 1   Issue id   33850 non-null  int64         
 2   Parent id  33850 non-null  int64         
 3   Created    33850 non-null  datetime64[ns]
 4   Resolved   33822 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 1.3+ MB


C:\Users\gmalarski\AppData\Local\Temp\ipykernel_4400\2151095576.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks["Resolved"] = pd.to_datetime(all_individual_tasks["Resolved"], dayfirst=True,  format='mixed')


In [308]:
all_individual_tasks['Summary'].value_counts()

Summary
Configure                                                                                   6148
Review                                                                                      5907
Communicate                                                                                 4970
Verify                                                                                      4893
Import to Test System                                                                       4405
Import in Production                                                                        4041
Import to Customer Test System                                                              1189
Import to Staging System                                                                     918
Explain                                                                                      752
Clarify                                                                                      406
Do                    

In [309]:
all_individual_tasks.sort_values(by='Created', ascending=False).head()

,Summary,Issue id,Parent id,Created,Resolved
32589,Clarify,8061841,8017846,2023-11-30 14:33:00,2023-11-30 14:44:00
32590,Verify,8061655,8061206,2023-11-30 14:00:00,2023-11-30 15:11:00
32591,Import to Test System,8061646,8061234,2023-11-30 13:59:00,2023-11-30 13:59:00
32592,Configure,8061645,8061234,2023-11-30 13:59:00,2023-11-30 13:59:00
32593,Import to Test System,8061617,8061206,2023-11-30 13:54:00,2023-11-30 14:00:00


### Check if there are any `null` values

In [310]:
all_individual_tasks.isnull().sum()

Summary       0
Issue id      0
Parent id     0
Created       0
Resolved     28
dtype: int64

### Drop all rows with null value

In [311]:
all_individual_tasks_without_null = all_individual_tasks.dropna()
all_individual_tasks_without_null.isnull().sum()

Summary      0
Issue id     0
Parent id    0
Created      0
Resolved     0
dtype: int64

### Identify and drop duplicated rows

In [312]:
all_individual_tasks_duplicated_rows = all_individual_tasks_without_null[all_individual_tasks_without_null.duplicated()]
all_individual_tasks_duplicated_rows

,Summary,Issue id,Parent id,Created,Resolved
32582,Review,7828796,7828795,2023-10-05 16:21:00,2023-11-02 14:35:00
33585,Communicate,7929412,7917819,2023-11-02 19:45:00,2023-11-02 19:45:00
33590,Import in Production,7928462,7877598,2023-11-02 15:58:00,2023-11-02 16:02:00
33591,Configure,7928460,7877598,2023-11-02 15:58:00,2023-11-02 15:58:00
33593,Communicate,7928316,7891707,2023-11-02 15:26:00,2023-11-02 15:37:00
...,...,...,...,...,...
33750,Configure,7892900,7811905,2023-10-23 16:34:00,2023-11-02 08:43:00
33763,Configure,7882479,7873215,2023-10-19 14:49:00,2023-11-02 13:55:00
33767,Review,7877599,7877598,2023-10-19 02:05:00,2023-11-02 15:58:00
33776,Communicate,7868731,7866065,2023-10-17 09:06:00,2023-11-02 10:49:00


In [313]:
all_individual_tasks_no_duplicates = all_individual_tasks_without_null.drop_duplicates()
all_individual_tasks_no_duplicates.sort_values(by='Created', ascending=False)

,Summary,Issue id,Parent id,Created,Resolved
32589,Clarify,8061841,8017846,2023-11-30 14:33:00,2023-11-30 14:44:00
32590,Verify,8061655,8061206,2023-11-30 14:00:00,2023-11-30 15:11:00
32591,Import to Test System,8061646,8061234,2023-11-30 13:59:00,2023-11-30 13:59:00
32592,Configure,8061645,8061234,2023-11-30 13:59:00,2023-11-30 13:59:00
32593,Import to Test System,8061617,8061206,2023-11-30 13:54:00,2023-11-30 14:00:00
...,...,...,...,...,...
1441,Review,7673004,7648948,2020-05-12 14:33:00,2021-01-08 15:47:00
3825,Review,7672973,7648940,2020-05-11 16:52:00,2021-04-06 09:19:00
3826,Clarify,7672796,7648837,2020-04-30 09:19:00,2021-03-03 09:21:00
1442,Review,7672645,7648888,2020-04-24 12:11:00,2021-01-21 15:53:00


In [314]:
all_individual_tasks_no_duplicates.sort_values(by='Created', ascending=False).head()

,Summary,Issue id,Parent id,Created,Resolved
32589,Clarify,8061841,8017846,2023-11-30 14:33:00,2023-11-30 14:44:00
32590,Verify,8061655,8061206,2023-11-30 14:00:00,2023-11-30 15:11:00
32591,Import to Test System,8061646,8061234,2023-11-30 13:59:00,2023-11-30 13:59:00
32592,Configure,8061645,8061234,2023-11-30 13:59:00,2023-11-30 13:59:00
32593,Import to Test System,8061617,8061206,2023-11-30 13:54:00,2023-11-30 14:00:00


## Summary

- We are interested only in completed tickets so we have droped rows which contain `null` in `resolved` columns
- Data is filtered and cleaned we are ready for analysis part

# Data Analysis 

# Daterminate how long each ticket was in particular individual task

- calculate time for eacch subtask
- calculate how long each ticket was in review
- calculate how long each ticket was "on codebusters"
- calculate how long each ticket was "outside codebusters"


## Calculate time spent in each of the subtask

In [315]:
all_individual_tasks_time_calculation = all_individual_tasks_no_duplicates
all_individual_tasks_time_calculation["time_in"] = all_individual_tasks_time_calculation["Resolved"] - all_individual_tasks_time_calculation["Created"]

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_4400\1582069957.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks_time_calculation["time_in"] = all_individual_tasks_time_calculation["Resolved"] - all_individual_tasks_time_calculation["Created"]


### List all available statuses of the CI ticket subtask

In [316]:
all_individual_tasks_time_calculation["Summary"].value_counts()

Summary
Configure                                                                                   6134
Review                                                                                      5873
Communicate                                                                                 4958
Verify                                                                                      4878
Import to Test System                                                                       4398
Import in Production                                                                        4037
Import to Customer Test System                                                              1179
Import to Staging System                                                                     918
Explain                                                                                      752
Clarify                                                                                      405
Do                    

In [317]:
all_individual_tasks_time_calculation.shape

(33749, 6)

In [318]:
all_individual_tasks_time_calculation[all_individual_tasks_time_calculation["Parent id"] == 7654073]

,Summary,Issue id,Parent id,Created,Resolved,time_in
26143,Explain,7706471,7654073,2023-06-14 08:36:00,2023-06-14 09:00:00,0 days 00:24:00


In [319]:
all_individual_tasks_time_calculation[all_individual_tasks_time_calculation["Parent id"] == 454905]["time_in"].sum()

Timedelta('0 days 00:00:00')

## Get the lates `communicate` subtaks date for the particular `Parent id`

In [320]:
all_individual_tasks_communicate_time = all_individual_tasks_time_calculation
all_individual_tasks_communicate_time = all_individual_tasks_communicate_time[all_individual_tasks_communicate_time['Summary'] == "Communicate"]
all_individual_tasks_communicate_time = all_individual_tasks_communicate_time.sort_values(["Parent id", "Created"], ascending=[True, False])
all_individual_tasks_communicate_time = all_individual_tasks_communicate_time.drop_duplicates(subset="Parent id", keep='first')
all_individual_tasks_communicate_time = all_individual_tasks_communicate_time.filter(items=["Parent id","Created"]).set_index('Parent id')
all_individual_tasks_communicate_time.rename({"Created": "Resolved-communicate"},axis=1, inplace=True)

all_individual_tasks_communicate_time

,Resolved-communicate
Parent id,
335272,2022-10-03 11:14:00
360791,2022-10-07 15:31:00
374097,2022-11-24 16:28:00
376147,2022-10-10 14:19:00
388122,2022-10-20 17:08:00
...,...
8046096,2023-11-28 12:36:00
8048208,2023-11-28 09:47:00
8049403,2023-11-29 11:29:00


In [321]:
all_individual_tasks_reivew = all_individual_tasks_time_calculation[all_individual_tasks_time_calculation['Summary'] == 'Review']
all_individual_tasks_reivew.rename({"time_in": "time_in_review"},axis=1, inplace=True)
all_individual_tasks_reivew.reset_index(inplace=True)
all_individual_tasks_reivew

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_4400\3555403275.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks_reivew.rename({"time_in": "time_in_review"},axis=1, inplace=True)


,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review
0,0,Review,7679692,7650050,2021-02-27 14:46:00,2021-02-27 14:46:00,0 days 00:00:00
1,16,Review,7679672,7650047,2021-02-26 14:28:00,2021-02-26 14:28:00,0 days 00:00:00
2,22,Review,7679664,7650045,2021-02-26 12:46:00,2021-02-26 13:17:00,0 days 00:31:00
3,33,Review,7679647,7650040,2021-02-25 22:41:00,2021-02-25 22:42:00,0 days 00:01:00
4,41,Review,7679633,7649967,2021-02-25 14:22:00,2021-02-25 16:31:00,0 days 02:09:00
...,...,...,...,...,...,...,...
5868,33838,Review,7706389,7654151,2023-06-12 13:30:00,2023-11-17 13:27:00,157 days 23:57:00
5869,33839,Review,7706388,7654150,2023-06-12 13:27:00,2023-11-17 13:25:00,157 days 23:58:00
5870,33840,Review,7706387,7654149,2023-06-12 13:26:00,2023-11-17 13:26:00,158 days 00:00:00
5871,33843,Review,7706264,7654141,2023-06-07 10:49:00,2023-11-17 12:28:00,163 days 01:39:00


In [322]:
all_individual_tasks_reivew.isnull().sum()

index             0
Summary           0
Issue id          0
Parent id         0
Created           0
Resolved          0
time_in_review    0
dtype: int64

In [323]:
all_sr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1348 entries, 0 to 1347
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   Issue key        1348 non-null   object         
 1   Issue id         1348 non-null   int64          
 2   Status           1348 non-null   object         
 3   Reporter         1348 non-null   object         
 4   Created          1348 non-null   datetime64[ns] 
 5   Due Date         1348 non-null   object         
 6   Resolved         1348 non-null   datetime64[ns] 
 7   Customer         1348 non-null   object         
 8   Type of Request  1348 non-null   object         
 9   Is Escalated     1348 non-null   bool           
 10  ticket_age       1348 non-null   timedelta64[ns]
dtypes: bool(1), datetime64[ns](2), int64(1), object(6), timedelta64[ns](1)
memory usage: 106.8+ KB


### Grup tickets by the `Parent id` and sum total time spent in `review` subtask


In [324]:
all_individual_tasks_reivew[all_individual_tasks_reivew["Parent id"] == 465210]

,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review


In [325]:
all_individual_tasks_reivew_sum = all_individual_tasks_reivew.groupby(by=["Parent id"], dropna=False)["time_in_review"].sum()
all_individual_tasks_reivew_sum = all_individual_tasks_reivew_sum.to_frame()
all_individual_tasks_reivew_sum.reset_index(inplace=True)
all_individual_tasks_reivew_sum

,Parent id,time_in_review
0,336670,0 days 00:01:00
1,393585,3 days 13:22:00
2,399228,52 days 06:54:00
3,402477,0 days 18:56:00
4,404450,11 days 19:36:00
...,...,...
5102,8055536,0 days 00:01:00
5103,8057074,0 days 16:43:00
5104,8060101,0 days 00:03:00
5105,8061206,0 days 00:13:00


In [326]:
all_individual_tasks_reivew_sum[all_individual_tasks_reivew_sum["Parent id"] == 465210]

,Parent id,time_in_review


In [327]:
all_individual_tasks_reivew_sum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5107 entries, 0 to 5106
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype          
---  ------          --------------  -----          
 0   Parent id       5107 non-null   int64          
 1   time_in_review  5107 non-null   timedelta64[ns]
dtypes: int64(1), timedelta64[ns](1)
memory usage: 79.9 KB


In [328]:
all_individual_tasks_reivew_sum[all_individual_tasks_reivew_sum["Parent id"] == 465210]

,Parent id,time_in_review


### Check on the particular ticket if the total time is summed correctly

In [329]:
all_individual_tasks_reivew.groupby("Parent id").count()

,index,Summary,Issue id,Created,Resolved,time_in_review
Parent id,,,,,,
336670,1,1,1,1,1,1
393585,1,1,1,1,1,1
399228,3,3,3,3,3,3
402477,2,2,2,2,2,2
404450,1,1,1,1,1,1
...,...,...,...,...,...,...
8055536,1,1,1,1,1,1
8057074,1,1,1,1,1,1
8060101,1,1,1,1,1,1


In [330]:
all_individual_tasks_reivew.loc[all_individual_tasks_reivew["Parent id"] == 345462]

,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review


In [331]:
all_individual_tasks_reivew_sum_df = all_individual_tasks_reivew_sum
all_individual_tasks_reivew_sum_df.reset_index(inplace=True)
all_individual_tasks_reivew_sum_df

,index,Parent id,time_in_review
0,0,336670,0 days 00:01:00
1,1,393585,3 days 13:22:00
2,2,399228,52 days 06:54:00
3,3,402477,0 days 18:56:00
4,4,404450,11 days 19:36:00
...,...,...,...
5102,5102,8055536,0 days 00:01:00
5103,5103,8057074,0 days 16:43:00
5104,5104,8060101,0 days 00:03:00
5105,5105,8061206,0 days 00:13:00


In [332]:
all_individual_tasks_reivew_sum_df.loc[all_individual_tasks_reivew_sum_df["Parent id"] == 345462]

,index,Parent id,time_in_review


In [333]:
all_individual_tasks_reivew_sum_df.isnull().sum()

index             0
Parent id         0
time_in_review    0
dtype: int64

In [334]:
all_individual_tasks_reivew_sum_df

,index,Parent id,time_in_review
0,0,336670,0 days 00:01:00
1,1,393585,3 days 13:22:00
2,2,399228,52 days 06:54:00
3,3,402477,0 days 18:56:00
4,4,404450,11 days 19:36:00
...,...,...,...
5102,5102,8055536,0 days 00:01:00
5103,5103,8057074,0 days 16:43:00
5104,5104,8060101,0 days 00:03:00
5105,5105,8061206,0 days 00:13:00


**Summary:**
- There are some tickets which have been more than once in the `Review` step
- We can notice that after gruping and summing tickets, case with 'Parent id' `345462` has total time equals more than `8 days`, which is the sum of the time spend in previous two `Review` subtasks

### Merge `all_sr` dataset with `all_individual_tasks_reivew_sum_df`

In [335]:
all_sr_review_time = pd.merge(all_sr, all_individual_tasks_reivew_sum_df[['Parent id', 'time_in_review']], how="left",left_on = "Issue id",right_on = "Parent id")
all_sr_review_time

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age,Parent id,time_in_review
0,CYTCI-10248143,7829513,Closed,askarja,2023-10-06 01:08:00,2023-10-13 00:00,2023-10-16 17:29:00,Porsche,Service Request (SR),False,10 days 16:21:00,7829513.0,7 days 15:23:00
1,CYTCI-10248142,7829500,Closed,askarja,2023-10-06 00:43:00,2023-10-13 00:00,2023-11-03 01:45:00,Porsche,Service Request (SR),False,28 days 01:02:00,7829500.0,11 days 08:33:00
2,CYTCI-10248129,7822247,Closed,yutaka.yasumi,2023-10-04 16:50:00,2023-10-14 00:00,2023-10-31 02:49:00,Hugo Boss,Service Request (SR),False,26 days 09:59:00,7822247.0,8 days 23:46:00
3,CYTCI-10248128,7819124,Closed,yutaka.yasumi,2023-10-04 05:35:00,2023-10-14 00:00,2023-10-04 07:14:00,MFTBC,Service Request (SR),False,0 days 01:39:00,7819124.0,0 days 01:39:00
4,CYTCI-10248127,7819049,Closed,yutaka.yasumi,2023-10-04 04:14:00,2023-10-13 00:00,2023-10-30 09:51:00,Hugo Boss,Service Request (SR),False,26 days 05:37:00,7819049.0,8 days 09:37:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1343,CYTCI-5864,7651300,Closed,yutaka.yasumi,2021-10-19 09:20:00,2022-01-27 00:00,2022-03-07 09:30:00,Continental,Service Request (SR),False,139 days 00:10:00,7651300.0,61 days 02:05:00
1344,CYTCI-5837,7651273,Closed,ssanz,2021-10-13 12:23:00,2021-10-17 00:00,2022-03-07 12:40:00,msg systems,Service Request (SR),True,145 days 00:17:00,7651273.0,1 days 23:58:00
1345,CYTCI-5749,7651186,Closed,kmarscholl,2021-09-29 08:51:00,2021-12-30 00:00,2022-01-19 17:20:00,OBB,Service Request (SR),False,112 days 08:29:00,7651186.0,14 days 00:03:00
1346,CYTCI-5527,7650968,Closed,ngraf,2021-08-06 14:47:00,2021-08-12 00:00,2022-01-13 16:01:00,School of Finance,Service Request (SR),True,160 days 01:14:00,7650968.0,26 days 02:43:00


In [336]:
all_sr_review_time.isnull().sum()

Issue key            0
Issue id             0
Status               0
Reporter             0
Created              0
Due Date             0
Resolved             0
Customer             0
Type of Request      0
Is Escalated         0
ticket_age           0
Parent id          244
time_in_review     244
dtype: int64

In [337]:
all_sr_review_time_without_null = all_sr_review_time.dropna()
all_sr_review_time_without_null.isnull().sum()

Issue key          0
Issue id           0
Status             0
Reporter           0
Created            0
Due Date           0
Resolved           0
Customer           0
Type of Request    0
Is Escalated       0
ticket_age         0
Parent id          0
time_in_review     0
dtype: int64

**Summary**:
- Ror tickets like `CI-8488`revew time equals 0, so we need to drop it, because it can leads to wrong KPIs calculations

# TO DO
- update the lates dateset

## Determinate time spent on codebusters

- calculate the total time ticket was in subtasks "Configure", "Review", "mport to Test System", "Import in Production", "Import to Customer Test System"
- apply the results to all tickets dataset

In [338]:
all_individual_with_codebusters_time = all_individual_tasks_time_calculation.loc[all_individual_tasks_time_calculation["Summary"].isin(["Configure", "Review", "Import to Test System", "Import in Production", "Import to Customer Test System", "Import to Test", "Import to Staging System"])]

Summarize time for each codebusters subtask 

In [339]:
all_individual_with_codebusters_time.groupby("Parent id").count()

,Summary,Issue id,Created,Resolved,time_in
Parent id,,,,,
336670,1,1,1,1,1
360791,1,1,1,1,1
376147,2,2,2,2,2
388122,1,1,1,1,1
392021,1,1,1,1,1
...,...,...,...,...,...
8055536,3,3,3,3,3
8057074,1,1,1,1,1
8060101,2,2,2,2,2


In [340]:
all_individual_with_codebusters_time[all_individual_with_codebusters_time['Parent id'] == 329036]

,Summary,Issue id,Parent id,Created,Resolved,time_in


In [341]:
all_individual_with_codebusters_time_sum = all_individual_with_codebusters_time.groupby("Parent id")["time_in"].sum()
all_individual_with_codebusters_time_sum_df = all_individual_with_codebusters_time_sum.to_frame()
all_individual_with_codebusters_time_sum_df.rename({"time_in": "time_on_codebusters"},axis=1, inplace=True)
all_individual_with_codebusters_time_sum_df.reset_index(inplace=True)
all_individual_with_codebusters_time_sum_df

,Parent id,time_on_codebusters
0,336670,0 days 00:01:00
1,360791,0 days 00:01:00
2,376147,0 days 00:00:00
3,388122,28 days 16:13:00
4,392021,232 days 01:49:00
...,...,...
5337,8055536,0 days 00:06:00
5338,8057074,0 days 16:43:00
5339,8060101,0 days 00:04:00
5340,8061206,0 days 01:14:00


In [342]:
# all_individual_tasks_reivew_sum_df = all_individual_tasks_reivew_sum.to_frame()
# all_individual_tasks_reivew_sum_df.reset_index(inplace=True)
# all_individual_tasks_reivew_sum_df

### Merge total time on codebusters with all jira ticekts dataset

In [343]:
all_sr_codebusters_time = pd.merge(all_sr, all_individual_with_codebusters_time_sum_df, how="inner",left_on = "Issue id",right_on = "Parent id")
all_sr_codebusters_time 

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age,Parent id,time_on_codebusters
0,CYTCI-10248143,7829513,Closed,askarja,2023-10-06 01:08:00,2023-10-13 00:00,2023-10-16 17:29:00,Porsche,Service Request (SR),False,10 days 16:21:00,7829513,10 days 15:06:00
1,CYTCI-10248142,7829500,Closed,askarja,2023-10-06 00:43:00,2023-10-13 00:00,2023-11-03 01:45:00,Porsche,Service Request (SR),False,28 days 01:02:00,7829500,11 days 08:33:00
2,CYTCI-10248129,7822247,Closed,yutaka.yasumi,2023-10-04 16:50:00,2023-10-14 00:00,2023-10-31 02:49:00,Hugo Boss,Service Request (SR),False,26 days 09:59:00,7822247,9 days 11:52:00
3,CYTCI-10248128,7819124,Closed,yutaka.yasumi,2023-10-04 05:35:00,2023-10-14 00:00,2023-10-04 07:14:00,MFTBC,Service Request (SR),False,0 days 01:39:00,7819124,0 days 01:39:00
4,CYTCI-10248127,7819049,Closed,yutaka.yasumi,2023-10-04 04:14:00,2023-10-13 00:00,2023-10-30 09:51:00,Hugo Boss,Service Request (SR),False,26 days 05:37:00,7819049,15 days 04:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124,CYTCI-5864,7651300,Closed,yutaka.yasumi,2021-10-19 09:20:00,2022-01-27 00:00,2022-03-07 09:30:00,Continental,Service Request (SR),False,139 days 00:10:00,7651300,90 days 12:39:00
1125,CYTCI-5837,7651273,Closed,ssanz,2021-10-13 12:23:00,2021-10-17 00:00,2022-03-07 12:40:00,msg systems,Service Request (SR),True,145 days 00:17:00,7651273,29 days 18:49:00
1126,CYTCI-5749,7651186,Closed,kmarscholl,2021-09-29 08:51:00,2021-12-30 00:00,2022-01-19 17:20:00,OBB,Service Request (SR),False,112 days 08:29:00,7651186,97 days 19:44:00
1127,CYTCI-5527,7650968,Closed,ngraf,2021-08-06 14:47:00,2021-08-12 00:00,2022-01-13 16:01:00,School of Finance,Service Request (SR),True,160 days 01:14:00,7650968,71 days 01:40:00


### Investigate `null` rows after merging  main SR tickets with individual subtasks

In [344]:
all_sr_codebusters_time.isnull().sum()

Issue key              0
Issue id               0
Status                 0
Reporter               0
Created                0
Due Date               0
Resolved               0
Customer               0
Type of Request        0
Is Escalated           0
ticket_age             0
Parent id              0
time_on_codebusters    0
dtype: int64

In [345]:
all_sr_codebusters_time_without_null = all_sr_codebusters_time.dropna()
all_sr_codebusters_time_without_null.isnull().sum()

Issue key              0
Issue id               0
Status                 0
Reporter               0
Created                0
Due Date               0
Resolved               0
Customer               0
Type of Request        0
Is Escalated           0
ticket_age             0
Parent id              0
time_on_codebusters    0
dtype: int64

In [346]:
all_sr_codebusters_time_without_null

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age,Parent id,time_on_codebusters
0,CYTCI-10248143,7829513,Closed,askarja,2023-10-06 01:08:00,2023-10-13 00:00,2023-10-16 17:29:00,Porsche,Service Request (SR),False,10 days 16:21:00,7829513,10 days 15:06:00
1,CYTCI-10248142,7829500,Closed,askarja,2023-10-06 00:43:00,2023-10-13 00:00,2023-11-03 01:45:00,Porsche,Service Request (SR),False,28 days 01:02:00,7829500,11 days 08:33:00
2,CYTCI-10248129,7822247,Closed,yutaka.yasumi,2023-10-04 16:50:00,2023-10-14 00:00,2023-10-31 02:49:00,Hugo Boss,Service Request (SR),False,26 days 09:59:00,7822247,9 days 11:52:00
3,CYTCI-10248128,7819124,Closed,yutaka.yasumi,2023-10-04 05:35:00,2023-10-14 00:00,2023-10-04 07:14:00,MFTBC,Service Request (SR),False,0 days 01:39:00,7819124,0 days 01:39:00
4,CYTCI-10248127,7819049,Closed,yutaka.yasumi,2023-10-04 04:14:00,2023-10-13 00:00,2023-10-30 09:51:00,Hugo Boss,Service Request (SR),False,26 days 05:37:00,7819049,15 days 04:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124,CYTCI-5864,7651300,Closed,yutaka.yasumi,2021-10-19 09:20:00,2022-01-27 00:00,2022-03-07 09:30:00,Continental,Service Request (SR),False,139 days 00:10:00,7651300,90 days 12:39:00
1125,CYTCI-5837,7651273,Closed,ssanz,2021-10-13 12:23:00,2021-10-17 00:00,2022-03-07 12:40:00,msg systems,Service Request (SR),True,145 days 00:17:00,7651273,29 days 18:49:00
1126,CYTCI-5749,7651186,Closed,kmarscholl,2021-09-29 08:51:00,2021-12-30 00:00,2022-01-19 17:20:00,OBB,Service Request (SR),False,112 days 08:29:00,7651186,97 days 19:44:00
1127,CYTCI-5527,7650968,Closed,ngraf,2021-08-06 14:47:00,2021-08-12 00:00,2022-01-13 16:01:00,School of Finance,Service Request (SR),True,160 days 01:14:00,7650968,71 days 01:40:00


**Summary:**

## Determinate time spent outside codebusters

- calculate the total time ticket was in one of the subtask "Explain", "Verify", "Clarify", "Communicate"
- apply results to all SR tickets dataset

In [347]:
all_individual_others_time = all_individual_tasks_time_calculation.loc[all_individual_tasks_time_calculation["Summary"].isin(["Explain", "Verify", "Clarify", "Communicate"])]
all_individual_others_time 

,Summary,Issue id,Parent id,Created,Resolved,time_in
1,Clarify,7679691,7650034,2021-02-26 17:29:00,2021-02-26 17:29:00,0 days 00:00:00
2,Explain,7679689,7650029,2021-02-26 15:37:00,2021-02-26 15:37:00,0 days 00:00:00
3,Communicate,7679687,7649877,2021-02-26 15:19:00,2021-02-26 15:20:00,0 days 00:01:00
5,Verify,7679685,7649877,2021-02-26 15:19:00,2021-02-26 15:19:00,0 days 00:00:00
9,Clarify,7679681,7649877,2021-02-26 15:18:00,2021-02-26 15:18:00,0 days 00:00:00
...,...,...,...,...,...,...
33815,Communicate,7838831,7826989,2023-10-09 15:07:00,2023-11-13 13:25:00,34 days 22:18:00
33820,Communicate,7709869,7654604,2023-09-27 12:49:00,2023-11-22 15:32:00,56 days 02:43:00
33828,Verify,7709616,7653979,2023-09-19 18:22:00,2023-11-06 09:34:00,47 days 15:12:00
33829,Verify,7709591,7654428,2023-09-19 12:53:00,2023-11-06 14:42:00,48 days 01:49:00


In [348]:
all_individual_others_time[all_individual_others_time["Parent id"] == 456393]

,Summary,Issue id,Parent id,Created,Resolved,time_in


Summarize time for each subtask outside codeubsters

In [349]:
all_individual_others_time_sum = all_individual_others_time.groupby("Parent id")["time_in"].sum()
all_individual_others_time_sum_df = all_individual_others_time_sum.to_frame()
all_individual_others_time_sum_df.rename({"time_in": "time_outside_codebusters"},axis=1, inplace=True)
all_individual_others_time_sum_df.reset_index(inplace=True)
all_individual_others_time_sum_df

,Parent id,time_outside_codebusters
0,335272,0 days 00:00:00
1,336670,3 days 15:01:00
2,338553,0 days 00:01:00
3,360791,0 days 00:00:00
4,374097,0 days 00:01:00
...,...,...
5250,8048691,0 days 00:01:00
5251,8049403,0 days 02:26:00
5252,8050182,0 days 00:00:00
5253,8055536,0 days 00:00:00


In [350]:
all_individual_others_time_sum_df[all_individual_others_time_sum_df["Parent id"] == 456393]

,Parent id,time_outside_codebusters


In [351]:
# all_individual_with_codebusters_time_sum = all_individual_with_codebusters_time.groupby("Parent id")["time_in"].sum()
# all_individual_with_codebusters_time_sum_df = all_individual_with_codebusters_time_sum.to_frame()
# all_individual_with_codebusters_time_sum_df.rename({"time_in": "time_on_codebusters"},axis=1, inplace=True)
# all_individual_with_codebusters_time_sum_df.reset_index(inplace=True)
# all_individual_with_codebusters_time_sum_df

### Merge `all_sr` with `all_individual_others_time_sum_df` dataset

In [352]:
all_sr_outside_codebusters_time = pd.merge(all_sr, all_individual_others_time_sum_df, how="inner",left_on = "Issue id",right_on = "Parent id")
all_sr_outside_codebusters_time

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age,Parent id,time_outside_codebusters
0,CYTCI-10248143,7829513,Closed,askarja,2023-10-06 01:08:00,2023-10-13 00:00,2023-10-16 17:29:00,Porsche,Service Request (SR),False,10 days 16:21:00,7829513,0 days 01:15:00
1,CYTCI-10248142,7829500,Closed,askarja,2023-10-06 00:43:00,2023-10-13 00:00,2023-11-03 01:45:00,Porsche,Service Request (SR),False,28 days 01:02:00,7829500,16 days 16:29:00
2,CYTCI-10248129,7822247,Closed,yutaka.yasumi,2023-10-04 16:50:00,2023-10-14 00:00,2023-10-31 02:49:00,Hugo Boss,Service Request (SR),False,26 days 09:59:00,7822247,16 days 22:07:00
3,CYTCI-10248128,7819124,Closed,yutaka.yasumi,2023-10-04 05:35:00,2023-10-14 00:00,2023-10-04 07:14:00,MFTBC,Service Request (SR),False,0 days 01:39:00,7819124,0 days 00:00:00
4,CYTCI-10248127,7819049,Closed,yutaka.yasumi,2023-10-04 04:14:00,2023-10-13 00:00,2023-10-30 09:51:00,Hugo Boss,Service Request (SR),False,26 days 05:37:00,7819049,11 days 00:38:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,CYTCI-5864,7651300,Closed,yutaka.yasumi,2021-10-19 09:20:00,2022-01-27 00:00,2022-03-07 09:30:00,Continental,Service Request (SR),False,139 days 00:10:00,7651300,48 days 11:31:00
1131,CYTCI-5837,7651273,Closed,ssanz,2021-10-13 12:23:00,2021-10-17 00:00,2022-03-07 12:40:00,msg systems,Service Request (SR),True,145 days 00:17:00,7651273,115 days 05:28:00
1132,CYTCI-5749,7651186,Closed,kmarscholl,2021-09-29 08:51:00,2021-12-30 00:00,2022-01-19 17:20:00,OBB,Service Request (SR),False,112 days 08:29:00,7651186,14 days 12:45:00
1133,CYTCI-5527,7650968,Closed,ngraf,2021-08-06 14:47:00,2021-08-12 00:00,2022-01-13 16:01:00,School of Finance,Service Request (SR),True,160 days 01:14:00,7650968,84 days 03:46:00


In [353]:
all_sr_outside_codebusters_time.isnull().sum()

Issue key                   0
Issue id                    0
Status                      0
Reporter                    0
Created                     0
Due Date                    0
Resolved                    0
Customer                    0
Type of Request             0
Is Escalated                0
ticket_age                  0
Parent id                   0
time_outside_codebusters    0
dtype: int64

**Summary:**

## Combine all created datasets
- In this step we will merge all created erlier datasets: `all_sr_outside_codebusters_time`, `all_sr_codebusters_time` and `all_sr_review_time` 

Merge df containing "review" time with df containg "time on codebusters" 

In [354]:
all_sr_time_on_and_outside_codebusters = pd.merge(all_sr_outside_codebusters_time, all_sr_codebusters_time[['Issue id','time_on_codebusters']],  how="inner",left_on = "Issue id",right_on = "Issue id")
all_sr_time_on_and_outside_codebusters

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters
0,CYTCI-10248143,7829513,Closed,askarja,2023-10-06 01:08:00,2023-10-13 00:00,2023-10-16 17:29:00,Porsche,Service Request (SR),False,10 days 16:21:00,7829513,0 days 01:15:00,10 days 15:06:00
1,CYTCI-10248142,7829500,Closed,askarja,2023-10-06 00:43:00,2023-10-13 00:00,2023-11-03 01:45:00,Porsche,Service Request (SR),False,28 days 01:02:00,7829500,16 days 16:29:00,11 days 08:33:00
2,CYTCI-10248129,7822247,Closed,yutaka.yasumi,2023-10-04 16:50:00,2023-10-14 00:00,2023-10-31 02:49:00,Hugo Boss,Service Request (SR),False,26 days 09:59:00,7822247,16 days 22:07:00,9 days 11:52:00
3,CYTCI-10248128,7819124,Closed,yutaka.yasumi,2023-10-04 05:35:00,2023-10-14 00:00,2023-10-04 07:14:00,MFTBC,Service Request (SR),False,0 days 01:39:00,7819124,0 days 00:00:00,0 days 01:39:00
4,CYTCI-10248127,7819049,Closed,yutaka.yasumi,2023-10-04 04:14:00,2023-10-13 00:00,2023-10-30 09:51:00,Hugo Boss,Service Request (SR),False,26 days 05:37:00,7819049,11 days 00:38:00,15 days 04:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,CYTCI-5864,7651300,Closed,yutaka.yasumi,2021-10-19 09:20:00,2022-01-27 00:00,2022-03-07 09:30:00,Continental,Service Request (SR),False,139 days 00:10:00,7651300,48 days 11:31:00,90 days 12:39:00
1111,CYTCI-5837,7651273,Closed,ssanz,2021-10-13 12:23:00,2021-10-17 00:00,2022-03-07 12:40:00,msg systems,Service Request (SR),True,145 days 00:17:00,7651273,115 days 05:28:00,29 days 18:49:00
1112,CYTCI-5749,7651186,Closed,kmarscholl,2021-09-29 08:51:00,2021-12-30 00:00,2022-01-19 17:20:00,OBB,Service Request (SR),False,112 days 08:29:00,7651186,14 days 12:45:00,97 days 19:44:00
1113,CYTCI-5527,7650968,Closed,ngraf,2021-08-06 14:47:00,2021-08-12 00:00,2022-01-13 16:01:00,School of Finance,Service Request (SR),True,160 days 01:14:00,7650968,84 days 03:46:00,71 days 01:40:00


In [355]:
all_sr_time_on_and_outside_codebusters.isnull().sum()

Issue key                   0
Issue id                    0
Status                      0
Reporter                    0
Created                     0
Due Date                    0
Resolved                    0
Customer                    0
Type of Request             0
Is Escalated                0
ticket_age                  0
Parent id                   0
time_outside_codebusters    0
time_on_codebusters         0
dtype: int64

### Merge `all_tickets_time_on_and_outside_codebusters` with `all_sr_review_time` dataset

In [356]:
all_sr_with_time = pd.merge(all_sr_time_on_and_outside_codebusters, all_sr_review_time[['Issue id','time_in_review']],  how="inner",left_on = "Issue id",right_on = "Issue id")
all_sr_with_time

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters,time_in_review
0,CYTCI-10248143,7829513,Closed,askarja,2023-10-06 01:08:00,2023-10-13 00:00,2023-10-16 17:29:00,Porsche,Service Request (SR),False,10 days 16:21:00,7829513,0 days 01:15:00,10 days 15:06:00,7 days 15:23:00
1,CYTCI-10248142,7829500,Closed,askarja,2023-10-06 00:43:00,2023-10-13 00:00,2023-11-03 01:45:00,Porsche,Service Request (SR),False,28 days 01:02:00,7829500,16 days 16:29:00,11 days 08:33:00,11 days 08:33:00
2,CYTCI-10248129,7822247,Closed,yutaka.yasumi,2023-10-04 16:50:00,2023-10-14 00:00,2023-10-31 02:49:00,Hugo Boss,Service Request (SR),False,26 days 09:59:00,7822247,16 days 22:07:00,9 days 11:52:00,8 days 23:46:00
3,CYTCI-10248128,7819124,Closed,yutaka.yasumi,2023-10-04 05:35:00,2023-10-14 00:00,2023-10-04 07:14:00,MFTBC,Service Request (SR),False,0 days 01:39:00,7819124,0 days 00:00:00,0 days 01:39:00,0 days 01:39:00
4,CYTCI-10248127,7819049,Closed,yutaka.yasumi,2023-10-04 04:14:00,2023-10-13 00:00,2023-10-30 09:51:00,Hugo Boss,Service Request (SR),False,26 days 05:37:00,7819049,11 days 00:38:00,15 days 04:59:00,8 days 09:37:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,CYTCI-5864,7651300,Closed,yutaka.yasumi,2021-10-19 09:20:00,2022-01-27 00:00,2022-03-07 09:30:00,Continental,Service Request (SR),False,139 days 00:10:00,7651300,48 days 11:31:00,90 days 12:39:00,61 days 02:05:00
1111,CYTCI-5837,7651273,Closed,ssanz,2021-10-13 12:23:00,2021-10-17 00:00,2022-03-07 12:40:00,msg systems,Service Request (SR),True,145 days 00:17:00,7651273,115 days 05:28:00,29 days 18:49:00,1 days 23:58:00
1112,CYTCI-5749,7651186,Closed,kmarscholl,2021-09-29 08:51:00,2021-12-30 00:00,2022-01-19 17:20:00,OBB,Service Request (SR),False,112 days 08:29:00,7651186,14 days 12:45:00,97 days 19:44:00,14 days 00:03:00
1113,CYTCI-5527,7650968,Closed,ngraf,2021-08-06 14:47:00,2021-08-12 00:00,2022-01-13 16:01:00,School of Finance,Service Request (SR),True,160 days 01:14:00,7650968,84 days 03:46:00,71 days 01:40:00,26 days 02:43:00


In [357]:
all_sr_with_time_communicate = all_sr_with_time.merge(all_individual_tasks_communicate_time, left_on="Issue id", right_on="Parent id", how="left")
all_sr_with_time_communicate.isnull().sum()

Issue key                     0
Issue id                      0
Status                        0
Reporter                      0
Created                       0
Due Date                      0
Resolved                      0
Customer                      0
Type of Request               0
Is Escalated                  0
ticket_age                    0
Parent id                     0
time_outside_codebusters      0
time_on_codebusters           0
time_in_review               25
Resolved-communicate        123
dtype: int64

In [358]:
all_sr_with_time_communicate[all_sr_with_time_communicate["Resolved-communicate"].isnull()]

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters,time_in_review,Resolved-communicate
3,CYTCI-10248128,7819124,Closed,yutaka.yasumi,2023-10-04 05:35:00,2023-10-14 00:00,2023-10-04 07:14:00,MFTBC,Service Request (SR),False,0 days 01:39:00,7819124,0 days 00:00:00,0 days 01:39:00,0 days 01:39:00,NaT
8,CYTCI-10248119,7812503,Closed,mmaciejczyk,2023-10-02 16:03:00,2023-10-09 00:00,2023-11-10 12:51:00,Rhenus,Service Request (SR),False,38 days 20:48:00,7812503,28 days 23:04:00,9 days 21:44:00,9 days 21:44:00,NaT
10,CYTCI-10248110,7809249,Closed,yutaka.yasumi,2023-10-02 05:43:00,2023-10-30 00:00,2023-10-02 05:43:00,Hugo Boss,Service Request (SR),False,0 days 00:00:00,7809249,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,NaT
12,CYTCI-10248108,7809219,Closed,yutaka.yasumi,2023-10-02 04:38:00,2023-10-11 00:00,2023-10-02 04:42:00,Hugo Boss,Service Request (SR),False,0 days 00:04:00,7809219,0 days 00:00:00,0 days 00:04:00,0 days 00:00:00,NaT
18,CYTCI-9539,7969256,Closed,askarja,2023-11-10 19:30:00,2023-11-17 00:00,2023-11-15 01:24:00,Porsche,Service Request (SR),False,4 days 05:54:00,7969256,1 days 10:52:00,2 days 19:02:00,2 days 19:02:00,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1077,CYTCI-6155,7651590,Closed,nsanchezruiz,2021-12-21 13:48:00,2021-12-28 00:00,2022-03-08 10:49:00,Amadeus,Service Request (SR),True,76 days 21:01:00,7651590,5 days 05:53:00,72 days 15:45:00,1 days 20:10:00,NaT
1083,CYTCI-6145,7651580,Closed,ssanz,2021-12-17 18:42:00,2021-12-22 00:00,2022-02-07 16:13:00,Rheinmetall,Service Request (SR),False,51 days 21:31:00,7651580,9 days 01:38:00,42 days 19:53:00,5 days 17:10:00,NaT
1097,CYTCI-6097,7651533,Closed,ssanz,2021-12-07 15:22:00,2021-12-13 00:00,2022-01-20 13:56:00,Allianz,Service Request (SR),False,43 days 22:34:00,7651533,35 days 20:19:00,8 days 02:15:00,7 days 02:46:00,NaT
1105,CYTCI-6034,7651470,Closed,guinoo,2021-11-19 06:13:00,2021-11-23 00:00,2022-01-04 07:27:00,Lufthansa City Center,Service Request (SR),False,46 days 01:14:00,7651470,38 days 18:04:00,7 days 07:10:00,7 days 07:10:00,NaT


### Merge `all_individual_tasks_communicate_time` with all tikets

Check for null values

In [359]:
all_sr_with_time.isnull().sum()

Issue key                    0
Issue id                     0
Status                       0
Reporter                     0
Created                      0
Due Date                     0
Resolved                     0
Customer                     0
Type of Request              0
Is Escalated                 0
ticket_age                   0
Parent id                    0
time_outside_codebusters     0
time_on_codebusters          0
time_in_review              25
dtype: int64

In [360]:
all_sr_with_time.shape

(1115, 15)

### Drop rows which contain null value except column `Resolved-communicate`

In [361]:
all_sr_with_time_without_null = all_sr_with_time_communicate.dropna(subset=all_sr_with_time_communicate.columns.difference(['Resolved-communicate']))
all_sr_with_time_without_null.isnull().sum()

Issue key                     0
Issue id                      0
Status                        0
Reporter                      0
Created                       0
Due Date                      0
Resolved                      0
Customer                      0
Type of Request               0
Is Escalated                  0
ticket_age                    0
Parent id                     0
time_outside_codebusters      0
time_on_codebusters           0
time_in_review                0
Resolved-communicate        118
dtype: int64

In [362]:
all_sr_with_time_without_null.shape

(1090, 16)

In [363]:
# all_sr_with_time_without_null.to_csv('all_sr.csv')

## Convert `time in..` columns to minutes

Convert `timedelta` columns to float

In [364]:
all_sr_with_time_converted = all_sr_with_time_without_null
all_sr_with_time_converted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1090 entries, 0 to 1114
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype          
---  ------                    --------------  -----          
 0   Issue key                 1090 non-null   object         
 1   Issue id                  1090 non-null   int64          
 2   Status                    1090 non-null   object         
 3   Reporter                  1090 non-null   object         
 4   Created                   1090 non-null   datetime64[ns] 
 5   Due Date                  1090 non-null   object         
 6   Resolved                  1090 non-null   datetime64[ns] 
 7   Customer                  1090 non-null   object         
 8   Type of Request           1090 non-null   object         
 9   Is Escalated              1090 non-null   bool           
 10  ticket_age                1090 non-null   timedelta64[ns]
 11  Parent id                 1090 non-null   int64          
 12  time_outsid

Convert `time in..` columns to minutes

In [365]:
all_sr_with_time_converted['time_in_review'] = all_sr_with_time_converted['time_in_review'].dt.total_seconds().div(60)
all_sr_with_time_converted['time_on_codebusters'] = all_sr_with_time_converted['time_on_codebusters'].dt.total_seconds().div(60)
all_sr_with_time_converted['time_outside_codebusters'] = all_sr_with_time_converted['time_outside_codebusters'].dt.total_seconds().div(60)
all_sr_with_time_converted['ticket_age'] = all_sr_with_time_converted['ticket_age'].dt.total_seconds().div(60)

all_sr_with_time_converted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1090 entries, 0 to 1114
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Issue key                 1090 non-null   object        
 1   Issue id                  1090 non-null   int64         
 2   Status                    1090 non-null   object        
 3   Reporter                  1090 non-null   object        
 4   Created                   1090 non-null   datetime64[ns]
 5   Due Date                  1090 non-null   object        
 6   Resolved                  1090 non-null   datetime64[ns]
 7   Customer                  1090 non-null   object        
 8   Type of Request           1090 non-null   object        
 9   Is Escalated              1090 non-null   bool          
 10  ticket_age                1090 non-null   float64       
 11  Parent id                 1090 non-null   int64         
 12  time_outside_codebusters 

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_4400\497592799.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sr_with_time_converted['time_in_review'] = all_sr_with_time_converted['time_in_review'].dt.total_seconds().div(60)
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_4400\497592799.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sr_with_time_converted['time_on_codebusters'] = all_sr_with_time_converted['time_on_codebusters'].dt.total_seconds().div(60)
C:\Users\gmalarski\AppData\Local\T

In [366]:
all_sr_with_time_converted.reset_index(inplace=True)
all_sr_with_time_converted

,index,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters,time_in_review,Resolved-communicate
0,0,CYTCI-10248143,7829513,Closed,askarja,2023-10-06 01:08:00,2023-10-13 00:00,2023-10-16 17:29:00,Porsche,Service Request (SR),False,15381.0,7829513,75.0,15306.0,11003.0,2023-10-16 16:14:00
1,1,CYTCI-10248142,7829500,Closed,askarja,2023-10-06 00:43:00,2023-10-13 00:00,2023-11-03 01:45:00,Porsche,Service Request (SR),False,40382.0,7829500,24029.0,16353.0,16353.0,2023-10-17 09:16:00
2,2,CYTCI-10248129,7822247,Closed,yutaka.yasumi,2023-10-04 16:50:00,2023-10-14 00:00,2023-10-31 02:49:00,Hugo Boss,Service Request (SR),False,38039.0,7822247,24367.0,13672.0,12946.0,2023-10-24 12:15:00
3,3,CYTCI-10248128,7819124,Closed,yutaka.yasumi,2023-10-04 05:35:00,2023-10-14 00:00,2023-10-04 07:14:00,MFTBC,Service Request (SR),False,99.0,7819124,0.0,99.0,99.0,NaT
4,4,CYTCI-10248127,7819049,Closed,yutaka.yasumi,2023-10-04 04:14:00,2023-10-13 00:00,2023-10-30 09:51:00,Hugo Boss,Service Request (SR),False,37777.0,7819049,15878.0,21899.0,12097.0,2023-10-27 12:15:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085,1110,CYTCI-5864,7651300,Closed,yutaka.yasumi,2021-10-19 09:20:00,2022-01-27 00:00,2022-03-07 09:30:00,Continental,Service Request (SR),False,200170.0,7651300,69811.0,130359.0,87965.0,2022-03-01 13:21:00
1086,1111,CYTCI-5837,7651273,Closed,ssanz,2021-10-13 12:23:00,2021-10-17 00:00,2022-03-07 12:40:00,msg systems,Service Request (SR),True,208817.0,7651273,165928.0,42889.0,2878.0,2022-03-07 09:51:00
1087,1112,CYTCI-5749,7651186,Closed,kmarscholl,2021-09-29 08:51:00,2021-12-30 00:00,2022-01-19 17:20:00,OBB,Service Request (SR),False,161789.0,7651186,20925.0,140864.0,20163.0,2022-01-13 13:51:00
1088,1113,CYTCI-5527,7650968,Closed,ngraf,2021-08-06 14:47:00,2021-08-12 00:00,2022-01-13 16:01:00,School of Finance,Service Request (SR),True,230474.0,7650968,121186.0,102340.0,37603.0,2022-01-13 16:01:00


In [367]:
all_sr_with_time_converted[all_sr_with_time_converted["Issue id"] == 456393]

,index,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters,time_in_review,Resolved-communicate


In [368]:
all_sr_with_time_converted['Created'].sort_values(ascending=False)

13     2023-11-22 12:45:00
14     2023-11-17 11:49:00
15     2023-11-15 17:11:00
16     2023-11-15 11:41:00
17     2023-11-14 09:17:00
               ...        
1085   2021-10-19 09:20:00
1086   2021-10-13 12:23:00
1087   2021-09-29 08:51:00
1088   2021-08-06 14:47:00
1089   2021-08-04 09:02:00
Name: Created, Length: 1090, dtype: datetime64[ns]

In [369]:
file_path = Path('../../DataSets/KPIs-final/all_sr_kpi.csv')
all_sr_with_time_converted.to_csv(path_or_buf=file_path)

# Investigation starts

In [370]:
all_individual_tasks_reivew[all_individual_tasks_reivew["Parent id"] == 438360]

,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review
3401,18969,Review,438363,438360,2022-12-29 11:45:00,2022-12-29 12:21:00,0 days 00:36:00
3402,18971,Review,438361,438360,2022-12-29 11:30:00,2022-12-29 11:42:00,0 days 00:12:00


In [371]:
all_sr[all_sr["Issue id"] == 438360]

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age


In [372]:
null_rows

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age,Parent id,time_in_review
13,CYTCI-9607,8018078,Closed,alejandro.ruizrobles,2023-11-22 12:45:00,2023-11-29 00:00,2023-11-29 16:42:00,Numiga,Service Request (SR),False,7 days 03:57:00,NaN,NaT
259,CYTCI-8680,7654073,Closed,alejandro.ruizrobles,2023-05-25 10:38:00,2023-06-01 00:00,2023-06-14 08:36:00,LKW Walter,Service Request (SR),False,19 days 21:58:00,NaN,NaT
537,CYTCI-7938,7653343,Closed,nsanchezruiz,2022-12-29 11:30:00,2022-12-29 00:00,2022-12-29 15:44:00,Ineos,Service Request (SR),False,0 days 04:14:00,NaN,NaT
538,CYTCI-7936,7653341,Closed,nsanchezruiz,2022-12-28 10:25:00,2022-12-31 00:00,2022-12-29 09:48:00,Daimler (MFTBC),Service Request (SR),False,0 days 23:23:00,NaN,NaT
540,CYTCI-7933,7653338,Closed,ssanz,2022-12-27 16:12:00,2023-01-02 00:00,2022-12-28 14:14:00,Allianz,Service Request (SR),False,0 days 22:02:00,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1341,CYTCI-5948,7651384,Closed,nsanchezruiz,2021-11-05 13:20:00,2021-11-25 00:00,2022-01-25 09:25:00,Allianz,Service Request (SR),False,80 days 20:05:00,NaN,NaT
1344,CYTCI-5837,7651273,Closed,ssanz,2021-10-13 12:23:00,2021-10-17 00:00,2022-03-07 12:40:00,msg systems,Service Request (SR),True,145 days 00:17:00,NaN,NaT
1345,CYTCI-5749,7651186,Closed,kmarscholl,2021-09-29 08:51:00,2021-12-30 00:00,2022-01-19 17:20:00,OBB,Service Request (SR),False,112 days 08:29:00,NaN,NaT
1346,CYTCI-5527,7650968,Closed,ngraf,2021-08-06 14:47:00,2021-08-12 00:00,2022-01-13 16:01:00,School of Finance,Service Request (SR),True,160 days 01:14:00,NaN,NaT


# Investigation ends

# To Do